In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a84efa1a9a637e072a34e350b82719e4f4159f8440c96ecc016bab072e0b3520
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-07 00:29:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.31MB/s    in 0.2s    

2023-02-07 00:29:17 (6.31 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://uoft-dabc-final-g5.s3.us-east-2.amazonaws.com/transformed_table.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("transformed_table.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
data_df.show()

+-------------------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|               date|fruits_veggies|daily_stress|places_visited|core_circle|supporting_others|social_network|achievement|donation|bmi_range|todo_completed|flow|daily_steps|live_vision|sleep_hours|lost_vacation|daily_shouting|sufficient_income|personal_awards|time_for_passion|weekly_meditation|       age|gender|work_life_balance_score|
+-------------------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+--------------

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.metrics import classification_report_imbalanced
import numpy as np
from pathlib import Path
from collections import Counter

In [ ]:
url = 'https://uoft-dabc-final-g5.s3.us-east-2.amazonaws.com/transformed_table.csv'
df = pd.read_csv(url)
df.head()

,date,fruits_veggies,daily_stress,places_visited,core_circle,supporting_others,social_network,achievement,donation,bmi_range,...,sleep_hours,lost_vacation,daily_shouting,sufficient_income,personal_awards,time_for_passion,weekly_meditation,age,gender,work_life_balance_score
0,2015-07-07,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,2015-07-07,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,2015-07-07,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,2015-07-07,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,2015-07-07,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


In [ ]:
# Drop the non-beneficial ID columns, 'date', 'bmi_range', 'age', 'gender'.
reduced_df = df.drop(['date', 'bmi_range', 'age', 'gender'], axis=1)
reduced_df.head()

,fruits_veggies,daily_stress,places_visited,core_circle,supporting_others,social_network,achievement,donation,todo_completed,flow,daily_steps,live_vision,sleep_hours,lost_vacation,daily_shouting,sufficient_income,personal_awards,time_for_passion,weekly_meditation,work_life_balance_score
0,3,2,2,5,0,5,2,0,6,4,5,0,7,5,5,1,4,0,5,609.5
1,2,3,4,3,8,10,5,2,5,2,5,5,8,2,2,2,3,2,6,655.6
2,2,3,3,4,4,10,3,2,2,2,4,5,8,10,2,2,4,8,3,631.6
3,3,3,10,3,10,7,2,5,3,5,5,0,5,7,5,1,5,2,0,622.7
4,5,1,3,3,10,4,2,4,5,0,5,0,7,0,0,2,8,1,5,663.9


In [ ]:
# Determine the number of unique values in each column.
reduced_df.nunique()

fruits_veggies                6
daily_stress                  6
places_visited               11
core_circle                  11
supporting_others            11
social_network               11
achievement                  11
donation                      6
todo_completed               11
flow                         11
daily_steps                  10
live_vision                  11
sleep_hours                  10
lost_vacation                11
daily_shouting               11
sufficient_income             2
personal_awards              11
time_for_passion             11
weekly_meditation            11
work_life_balance_score    1696
dtype: int64

In [ ]:
# Look at work_life_balance_score value counts for binning
WLB_count = reduced_df.work_life_balance_score.value_counts()
WLB_count

641.4    75
660.5    57
670.7    37
696.4    37
675.8    36
         ..
794.8     1
562.1     1
555.6     1
531.0     1
790.5     1
Name: work_life_balance_score, Length: 1696, dtype: int64

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_WLB = list(WLB_count.index)

# Replace in dataframe
for amt in replace_WLB:
    if amt <= 700:
        reduced_df.work_life_balance_score = reduced_df.work_life_balance_score.replace(amt,str("needs_improvement"))
    else:
        reduced_df.work_life_balance_score = reduced_df.work_life_balance_score.replace(amt,str("excellent"))

# Check to make sure binning was successful
reduced_df.work_life_balance_score.value_counts()

needs_improvement    12185
excellent             3786
Name: work_life_balance_score, dtype: int64

In [ ]:
# Split our preprocessed data into our features and target arrays
y = reduced_df["work_life_balance_score"].values
X = reduced_df.drop(["work_life_balance_score"], axis=1)
# .values returns a numpy array, not a Pandas dataframe. An array does not have a columns attribute

In [ ]:
# Split into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, stratify=y)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 823,  124],
       [  95, 2951]])

In [ ]:
# Display the accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9451540195341849

In [ ]:
# Print Classification Report
report = classification_report(y_test, y_pred)
print(report)

                   precision    recall  f1-score   support

        excellent       0.90      0.87      0.88       947
needs_improvement       0.96      0.97      0.96      3046

         accuracy                           0.95      3993
        macro avg       0.93      0.92      0.92      3993
     weighted avg       0.94      0.95      0.94      3993



In [ ]:
# Split our preprocessed data into our features and target arrays removing low importance
minimal_df = reduced_df.drop(['daily_stress', 'flow', 'lost_vacation', 'social_network', 'daily_shouting', 'sufficient_income', 'sleep_hours'], axis=1)

y = minimal_df["work_life_balance_score"].values
X = minimal_df.drop(["work_life_balance_score"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)
Counter(y_train)

Counter({'needs_improvement': 9139, 'excellent': 2839})

In [ ]:
# Logistic Regression
minimal_model = LogisticRegression(solver='lbfgs', random_state=1)
minimal_model = minimal_model.fit(X_train, y_train)

In [ ]:
# Display the confusion matrix
y_pred = minimal_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 725,  222],
       [ 144, 2902]])

In [ ]:
# Display the accuracy score
accuracy_score(y_test, y_pred)

0.9083395942900075

In [ ]:
# Print Classification Report
report = classification_report(y_test, y_pred)
print(report)

                   precision    recall  f1-score   support

        excellent       0.83      0.77      0.80       947
needs_improvement       0.93      0.95      0.94      3046

         accuracy                           0.91      3993
        macro avg       0.88      0.86      0.87      3993
     weighted avg       0.91      0.91      0.91      3993



In [ ]:
# As per StackOverflow, there is no feature importances ranking attribute for Logistic Regression.